## Run CNMF source extraction on movies
Step 2 of the Caiman processing pipeline for dendritic two-photon calcium imaging movies. This part uses mmap files as input. These are created during motion correction with the Caiman toolbox (see `01_Preprocess_MotionCorrect.ipynb`). 

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [ ]:
# Generic imports
# from __future__ import absolute_import, division, print_function
# from builtins import *

import os, platform, glob, sys, re
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat

from IPython.display import clear_output

# Import Bokeh library
from bokeh.plotting import Figure, show
from bokeh.models import Range1d, CrosshairTool, HoverTool, Legend
from bokeh.io import output_notebook
from bokeh.io import export_svgs

%matplotlib inline

In [ ]:
# on Linux we have to add the caiman folder to Pythonpath
if platform.system() == 'Linux':
    sys.path.append(os.path.expanduser('~/caiman'))
# environment variables for parallel processing
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['VECLIB_MAXIMUM_THREADS']='1'

In [ ]:
# CaImAn imports
import caiman as cm
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.components_evaluation import estimate_components_quality as estimate_q
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.source_extraction.cnmf import utilities as cnmf_utils

### Select files and parameters
The following need to be specified:
- data_folder ... location of the data (relative to ~/Data)
- mc_output ... select if output of rigid ('rig') or piece-wise rigid ('els') motion correction should be used (currently only 'rig' is tested and works)
- max_files ... maximum number of files to process, e.g. for testing (if 0, all files will be processed)

In [ ]:
animal_folder = 'M1_for_processing'
date_folder = 'M1_2018-01-31'
session_folder = 'S1'

mc_output = 'rig'
remove_bad_frames = True # remove bad frames specified in Json file

# create the complete path to the data folder
if platform.system() == 'Linux':
    data_folder = '/home/ubuntu/Data/Henry_test'
elif platform.system() == 'Darwin':
    data_folder = '/Users/Henry/polybox/Data_temp/Dendrites_Gwen'
data_folder = os.path.join(data_folder, animal_folder, date_folder, session_folder)
data_folder

In [ ]:
# select the mmap file created during motion correction
all_files = os.listdir(data_folder)
matches = [x for x in all_files if x.startswith('%s_%s' % (date_folder, session_folder)) 
           and x.endswith('.mmap') and mc_output in x]

if len(matches) == 1: # if only 1 file matches, select it
    fname = matches[0]
    print('Selected file: %s' % (fname))
elif len(matches) > 0: # if several files match, ask which one to choos
    print('Found several matching files!')
    for ix, match in enumerate(matches):
        print('%1.0f: %s' % (ix+1,match))
    select = raw_input('Please select your file (1-%1.0f):' % (ix+1))
    fname = matches[int(select)-1]
    print('Selected file: %s' % (fname))
else: # if no file matches, throw an error
    raise Exception('Did not find a matching mmap file!')

fname = os.path.join(data_folder, fname)

# get metadata for corresponding joined file
meta_fname = fname[:fname.find('_crop_')+5] + '.json'
print('Metadata: %s' % (meta_fname))
meta = json.load(open(meta_fname))

In [ ]:
len(meta['source_file'])

### Setup cluster
The default backend mode for parallel processing is through the multiprocessing package. This will allow us to use all the cores in the VM.

In [ ]:
# start the cluster (if a cluster already exists terminate it)
n_processes = 4 # number of compute processes (None to select automatically)
if 'dview' in locals():
    dview.terminate()
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)

### Parameters for source extraction
Next, we define the important parameters for calcium source extraction. These parameters will have to be iteratively refined for the respective datasets.

The parameters are stored in the Python dictionary `params_cnmf`.

In [ ]:
# parameters for source extraction and deconvolution
p = 1                       # order of the autoregressive system (normally 1, 2 for fast indicators / imaging)
gnb = 2                     # number of global background components
merge_thresh = 0.8          # merging threshold, max correlation allowed
rf = None                   # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50 / None: no patches
stride_cnmf = 6             # amount of overlap between the patches in pixels
K = 20                      # number of components per patch
gSig = [4, 4]               # expected half size of neurons
init_method = 'sparse_nmf'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
is_dendrites = True         # flag for analyzing dendritic data
#alpha_snmf = 10e2           # sparsity penalty for dendritic data analysis through sparse NMF
alpha_snmf = 1e-6

method_deconvolution='oasis'# deconvolution method (oasis or cvxpy)
final_rate = 13.1316             # final frame rate in Hz


# parameters for component evaluation
# min_SNR = 2.5               # signal to noise ratio for accepting a component
# rval_thr = 0.8              # space correlation threshold for accepting a component
# cnn_thr = 0.8               # threshold for CNN based classifier

In [ ]:
# store parameters in Python dictionary
params = dict()
params['p'] = p
params['gnb'] = gnb
params['merge_thresh'] = merge_thresh
params['rf'] = rf
params['stride_cnmf'] = stride_cnmf
params['K'] = K
params['gSig'] = gSig
params['init_method'] = init_method
params['is_dendrites'] = is_dendrites
params['alpha_snmf'] = alpha_snmf

params['method_deconvolution'] = method_deconvolution
params['final_rate'] = final_rate

params

### Run CNMF on joined mmap file
According to the Caiman tutorials, CNMF source extraction should be run in several stages. First, we run CNMF on patches of the dataset and then evaluate the quality of the extracted components. Next, CNMF is run again but on the full field-of-view. Finally, the extracted components are again evaluated and classified (good and bad).

Load mmap file.

In [ ]:
Yr, dims, T = cm.load_memmap(fname)

In [ ]:
fname

In [ ]:
# remove bad frames specified in corresponding Json file
# Todo: print deleted frames
if remove_bad_frames:
    bad_frames = json.load(open(fname.replace('.mmap','badFrames.json')))
    bad_frames = np.array(bad_frames['frames'])
    Yr = np.delete(Yr, bad_frames, axis=1)
    T = Yr.shape[1]
    images = np.reshape(Yr.T, [T] + list(dims), order='F')
    fname_new = cm.save_memmap([images], base_name=os.path.join(data_folder, 'removedFrames'))
    Yr, dims, T = cm.load_memmap(fname_new)
    print('Deleted %1d frames. Saved to new file %s.' % (len(bad_frames), os.path.basename(fname_new)))
    print('Deleted frames:')
    print(bad_frames)

In [ ]:
d1, d2 = dims
images = np.reshape(Yr.T, [T] + list(dims), order='F')
Y = np.reshape(Yr, dims + (T,), order='F')
m_images = cm.movie(images)
#  checks on movies (might take time if large!)
if np.min(images) < 0:
    raise Exception('Movie too negative, add_to_movie should be larger')
if np.sum(np.isnan(images)) > 0:
    raise Exception('Movie contains nan! You did not remove enough borders')
# correlation image
Cn = cm.local_correlations(Y)
Cn[np.isnan(Cn)] = 0

In [ ]:
# sanity check the image
avg_img = np.mean(images,axis=0)
plt.figure(figsize=(10,20))
plt.subplot(1,2,1)
plt.imshow(avg_img, cmap='gray'), plt.title('Frame average')
plt.subplot(1,2,2)
plt.imshow(Cn, cmap='jet'), plt.title('Correlation image')

### Run single CNMF on full FoV, without patches

In [ ]:
# Configure CNMF and fit to the data
cnmf_single = cnmf.CNMF(n_processes, k=params['K'], gSig=params['gSig'], merge_thresh=params['merge_thresh'], 
                p=params['p'], dview=dview, Ain=None, gnb=params['gnb'], rf=params['rf'],
                memory_fact=1, method_init=params['init_method'], alpha_snmf=params['alpha_snmf'],
                method_deconvolution=params['method_deconvolution'])
cnmf_single = cnmf_single.fit(images)

In [ ]:
# Unravel results
A, C, b, f, YrA, sn = cnmf_single.A, cnmf_single.C, cnmf_single.b, cnmf_single.f, cnmf_single.YrA, cnmf_single.sn
# A   ... n_pixel x n_components sparse matrix (component locations)
# C   ... n_component x t np.array (fitted signal)
# b   ... ? np.array
# f   ... ? np.array (b / f related to global background components)
# YrA ... n_component x t np.array (residual)
# sn  ... n_pixel np.array (SNR?)

print('Detected components: %1.0f' % (C.shape[0]))

Evaluate components (discard low quality components).

In [ ]:
r_values_min = .65         # threshold on space consistency
fitness_min = -100        # threshold on time variability
fitness_delta_min = -100  # threshold on time variability (if nonsparse activity)
Npeaks = 100
thresh_C = 0.3
Athresh = 0.1

traces = C + YrA
idx_comps, idx_comps_bad, fitness_raw, fitness_delta, r_values = estimate_q(traces, Y, A, C, b, f, final_frate=params['final_rate'], 
                                      Npeaks=Npeaks, r_values_min=r_values_min, fitness_min=fitness_min, 
                                      fitness_delta_min=fitness_delta_min, thresh_C=thresh_C, Athresh=Athresh,
                                      return_all=True)
print('Good / bad components: %1.0f / %1.0f' % (len(idx_comps), len(idx_comps_bad)))

In [ ]:
# Plot good and bad components
plt.figure(figsize=(20,30));
plt.subplot(121); crd_good = cm.utils.visualization.plot_contours(A[:,idx_comps], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of accepted components')
plt.subplot(122); crd_bad = cm.utils.visualization.plot_contours(A[:,idx_comps_bad], Cn, thr=.8, vmax=0.75)
plt.title('Contour plots of rejected components')

In [ ]:
# Plot good components on background image and as component map
A_dense = A.todense()
counter = 1
plt.figure(figsize=(20,20));
for i_comp in range(len(idx_comps)):
    plt.subplot(len(idx_comps),2,counter)
    counter += 1
    dummy = cm.utils.visualization.plot_contours(A[:,idx_comps[i_comp]], avg_img, cmap='gray', 
                                                 colors='r', display_numbers=False)
    component_img = np.array(np.reshape(A_dense[:,idx_comps[i_comp]], dims, order='F'))
    plt.subplot(len(idx_comps),2,counter)
    counter += 1
    plt.imshow(component_img), plt.title('Component %1.0f' % (i_comp))
    
    if i_comp == 0:
        component_matrix = component_img
    else:
        component_matrix = np.dstack((component_matrix, component_img))
plt.tight_layout()

### Save and view results

In [ ]:
# saving
np.savez(os.path.join(os.path.split(fname_new)[0],
                      os.path.split(fname_new)[1][:-5] + 'results_CNMF.npz'),
         Cn=Cn, A=A.todense(), C=C,
         b=b, f=f, YrA=YrA, sn=sn, d1=d1, d2=d2,
         idx_components=idx_comps, idx_components_bad=idx_comps_bad)

Interactive viewer for traces of accepted and rejected components
**Currently does not work remotely!**

In [ ]:
# accepted components
if len(idx_comps) > 0:
    nb_view_patches(Yr, A.tocsc()[:, idx_comps], C[idx_comps], 
                    b, f, dims[0], dims[1], YrA=YrA[idx_comps], image_neurons = Cn,
                    denoised_color = 'red');
else:
    print("No accepted components!")

### Calculate DFF and plot traces
The CaImAn function `detrend_df_f` uses a sliding window percentile filter to determine the baseline and compute DFF. 

In [ ]:
F_dff = cnmf_utils.detrend_df_f(A, b, C, f, YrA = YrA, quantileMin=8, frames_window=100)
# select good components
F_dff = F_dff[idx_comps,:]

t = np.arange(0, F_dff.shape[-1]) / params['final_rate']

For plotting we use the Bokeh interactive visualization library, which offers a number of nice features for plotting and interacting with plots.

In [ ]:
# This has to be in a separate cell, otherwise it wont work.
from bokeh import resources
output_notebook(resources=resources.INLINE)

Define some functions for plotting.

In [ ]:
def getHover():
    """Define and return hover tool for a plot"""
    # Define hover tool
    hover = HoverTool()
    hover.tooltips = [
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
    ]
    return hover

In [ ]:
def plotTimeseries(p, t, y, legend=None, stack=True, xlabel='', ylabel='', output_backend='canvas'):
    """
    Plot a timeseries in Figure p using the Bokeh library
    
    Input arguments:
    p ... Bokeh figure
    t ... 1d time axis vector (numpy array)
    y ... 2d data numpy array (number of traces x time)
    legend ... list of items to be used as figure legend
    stack ... whether to stack traces or nor (True / False)
    xlabel ... label for x-axis
    ylabel ... label for y-axis
    output_backend ... 'canvas' or 'svg'
    """
    
    colors_list = ['red', 'green', 'blue', 'yellow', 'cyan', 'orange', 'magenta', 'black', 'gray']
    p.add_tools(CrosshairTool(), getHover())
    
    offset = 0
    for i in range(y.shape[0]):
        if len(colors_list) < i+1:
            colors_list = colors_list + colors_list
        
        plot_trace = y[i, :]
        if stack:
            plot_trace = plot_trace - min(plot_trace) + offset
            offset = max(plot_trace)
        
        p.line(t, plot_trace, line_width=2, legend=legend[i], color=colors_list[i])
        
#     p.legend.location = (0,-30)
    p.legend.click_policy="hide"
    
    # format plot
    p.xaxis.axis_label = xlabel
    p.yaxis.axis_label = ylabel
    
    p.x_range = Range1d(np.min(t), np.max(t))
    
    p.background_fill_color = None
    p.border_fill_color = None
    
    p.output_backend = output_backend

    show(p)
    
    return p

The next cell plots the figure. The function `plotTimeseries` can plot traces on top of each other (`stack=False`) or stacked (`stack=True`). The interactive toolbar on the right of the figures allows panning, zooming, saving etc. One can also hide traces by clicking the corresponding legend item. To save the figure, click the disk icon in the plotting toolbar. With the default `output_backend` ('canvas'), a png file will be saved. To save to svg format, change `output_backend` to 'svg'.

In [ ]:
p = Figure(plot_width=800, plot_height=300, 
           title=('%s %s CNMF Results' % (date_folder, session_folder)))    
legend_text = ['Component %1d' % (x) for x in range(len(idx_comps))]
# this is the call to the plotting function (change args. as required)
plotTimeseries(p, t, F_dff, 
               legend=legend_text, 
               stack=True, 
               xlabel='Time [s]', ylabel='DFF',
               output_backend='canvas'
              )

### Split up by trials and save as .mat

In [ ]:
# split up F_dff by trials
# Todo: correct source_frames: key
source_files = meta['source_file']
source_frames = np.array(meta['source_frames'])

In [ ]:
# check if our numbers match
if not (np.sum(source_frames)-len(bad_frames)) == F_dff.shape[-1]:
    raise Exception('Sum of source frames minus number of bad frames must be equal to number of timepoints.')

In [ ]:
results_dff = dict()
removed_frames = dict()
bad_frames_removed = []
start_frame = 0
for ix, trial_file in enumerate(source_files):
    trial_indices = list(range(start_frame, source_frames[ix] + start_frame))
#     print(trial_file)
#     print('Start / stop: %1.0f / %1.0f' % (trial_indices[0], trial_indices[-1]))
    
    # check if trial contains bad frames (that have not been removed yet)
    bad = [x for x in bad_frames if x in trial_indices and x not in bad_frames_removed]
    
    # remove indices of bad frames
    if len(bad) > 0:
#         print('Found %1.0f bad frames' % (len(bad)))
        del trial_indices[-len(bad):]
#         print('Removed frames (relative to trial start):')
        for i_bad in bad:
            bad_frames_removed.append(i_bad)
        removed_frames_trial = [x-trial_indices[0] for x in bad]
    else:
        removed_frames_trial = []
    
#     print('Start / stop: %1.0f / %1.0f\n' % (trial_indices[0], trial_indices[-1]))
    
    # create a valid Matlab variable / field name
    field_name = str('x' + source_files[ix]).replace('.tif','').replace('-','_')
    results_dff[field_name] = F_dff[:,trial_indices]
    removed_frames[field_name] = removed_frames_trial
    
    start_frame = trial_indices[-1] + 1

In [ ]:
# prepare the dictionary for saving as mat file
# the field names will be the variable names in Matlab
mdict={'trials': [str(x) for x in source_files], 
       'dff_trial': results_dff, 
       'removed_frames': removed_frames,
       'mean_image': avg_img,
       'spatial_components': component_matrix
      }

In [ ]:
# save the .mat file
matfile_name = '%s_%s_CNMF_results.mat' % (date_folder, session_folder)
savemat(os.path.join(data_folder, matfile_name), mdict=mdict)